# 第四章：Layer 1 金属层检测 — `LayerParser` 详解

## 物理问题背景

### 为什么需要检测金属层？

在金属-水界面分析中，我们需要知道：
1. **金属板有几层？** — 了解系统结构
2. **哪两层是界面层？** — 只有这两层直接接触水
3. **界面层的法向量是什么方向？** — 决定「从哪里出发」测量密度

### 法向量的物理含义

**法向量**（normal vector）是垂直于金属表面的方向向量，在本项目中代表**金属→水的方向**：

```
                +z
                ↑
  液态水 (上)   │      ← normal_unit = (0, 0, +1) 朝上
  ─────────────┼──────   （high_c 界面层的法向量）
  Cu 层 3      │
  Cu 层 2      │
  Cu 层 1      │
  Cu 层 0      │
  ─────────────┼──────   （low_c 界面层的法向量）
  液态水 (下)   │      ← normal_unit = (0, 0, -1) 朝下
                │
```

`detect_interface_layers` 函数就是自动完成以上识别的工具。

In [ ]:
import sys
from pathlib import Path

PROJECT_ROOT = Path("..").resolve()
sys.path.insert(0, str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data_example" / "potential"
OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(exist_ok=True)

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ase.io

from src.structure.Analysis.WaterAnalysis._common import _parse_abc_from_md_inp

# 读取第一帧并设置晶胞
md_inp_path = DATA_DIR / "md.inp"
xyz_path    = DATA_DIR / "md-pos-1.xyz"

a_A, b_A, c_A = _parse_abc_from_md_inp(md_inp_path)
atoms = ase.io.read(str(xyz_path), index=0)
atoms.set_cell([a_A, b_A, c_A])
atoms.set_pbc([True, True, True])

print(f"读取第一帧：{len(atoms)} 个原子，晶胞 c = {c_A:.3f} Å")

---

## 4.1 调用 `detect_interface_layers`

In [ ]:
from src.structure.utils import detect_interface_layers, format_detection_summary

# 检测界面层
检测结果 = detect_interface_layers(atoms)

# 用内置函数打印摘要
print(format_detection_summary(检测结果))

### 解读 `SurfaceDetectionResult` dataclass

`detect_interface_layers` 返回一个 `SurfaceDetectionResult` 对象，包含三个字段：

In [ ]:
print("SurfaceDetectionResult 的字段：")
print()

# 字段1：axis_unit — 用于排序金属层的轴方向（单位向量）
print(f"1. axis_unit（排序轴方向，单位向量）:")
print(f"   {检测结果.axis_unit}")
print(f"   含义：z 方向 (0, 0, 1) → 沿 z 轴从低到高排列各层")
print()

# 字段2：metal_indices — 所有金属原子的编号
print(f"2. metal_indices（所有金属原子编号）:")
print(f"   共 {len(检测结果.metal_indices)} 个金属原子")
print(f"   前10个编号: {检测结果.metal_indices[:10]}")
print()

# 字段3：metal_layers_sorted — 所有金属层（按 z 坐标从低到高排列）
print(f"3. metal_layers_sorted（金属层，按 z 排序）:")
print(f"   共 {len(检测结果.metal_layers_sorted)} 层")

### `Layer` dataclass 详解

每个 `Layer` 对象代表一层金属原子，用表格展示所有层的信息：

In [ ]:
all_layers = 检测结果.metal_layers_sorted

# 建立表格
表格数据 = []
for i, layer in enumerate(all_layers):
    normal_str = str(layer.normal_unit) if layer.normal_unit else '无（非界面层）'
    表格数据.append({
        '层编号': i,
        '层中心 center_s (Å)': round(layer.center_s, 3),
        '原子数': len(layer.atom_indices),
        '是界面层 is_interface': '✅ 是' if layer.is_interface else '❌ 否',
        '法向量 normal_unit': normal_str,
    })

df = pd.DataFrame(表格数据)
print(df.to_string(index=False))

### 可视化：各层位置 + 界面层标注

在原子 z 坐标散点图上，标注每个金属层的位置，区分界面层和非界面层。

In [ ]:
# 获取所有原子的 z 坐标和元素
所有坐标 = atoms.get_positions()
所有元素 = np.array(atoms.get_chemical_symbols())
z坐标 = 所有坐标[:, 2]

金属掩码 = np.isin(所有元素, ['Cu', 'Ag'])
水O掩码  = (所有元素 == 'O')

fig, ax = plt.subplots(figsize=(10, 6))

# 画金属原子（用 x 坐标做散点，主要展示 z 坐标分布）
x坐标 = 所有坐标[:, 0]  # x 方向坐标
ax.scatter(z坐标[金属掩码], x坐标[金属掩码], s=4, color='tab:orange', alpha=0.6, label='金属原子 Cu/Ag')
ax.scatter(z坐标[水O掩码],  x坐标[水O掩码],  s=4, color='tab:blue',   alpha=0.4, label='水分子 O')

# 标注每个金属层的中心位置
for i, layer in enumerate(all_layers):
    if layer.is_interface:
        # 界面层：红色竖线 + 箭头表示法向量
        ax.axvline(x=layer.center_s, color='red', linewidth=2.5, linestyle='--', alpha=0.8)
        ax.text(layer.center_s, ax.get_ylim()[1] if ax.get_ylim()[1] != 1.0 else a_A * 0.9,
                f'\n界面层 {i}\n↕ {layer.normal_unit[2]:+.0f}z',
                ha='center', color='red', fontsize=9, fontweight='bold')
    else:
        # 非界面层：蓝色竖线
        ax.axvline(x=layer.center_s, color='royalblue', linewidth=1.5, linestyle=':', alpha=0.6)
        ax.text(layer.center_s, 0.5,
                f'层{i}', ha='center', color='royalblue', fontsize=8)

ax.set_xlabel('z 坐标 (Å)  ←  按 z 轴展开', fontsize=12)
ax.set_ylabel('x 坐标 (Å)', fontsize=12)
ax.set_title('金属层检测结果（第一帧）\n红虚线 = 界面层，蓝点线 = 内部层', fontsize=13)
ax.legend(fontsize=10, loc='upper left')

plt.tight_layout()
plt.show()

print("\n界面层总结：")
for layer in 检测结果.interface_layers():
    z方向 = '+z (朝上，指向上侧水)' if layer.normal_unit[2] > 0 else '-z (朝下，指向下侧水)'
    print(f"  center_s = {layer.center_s:.2f} Å, 法向量方向: {z方向}")

---

## 4.2 算法内部逻辑（逐步演示）

让我们手动重现 `detect_interface_layers` 内部的每个步骤，理解算法。

### 步骤 1：筛选金属原子

In [ ]:
from src.structure.utils.config import DEFAULT_METAL_SYMBOLS

# 筛选金属原子
元素数组 = np.array(atoms.get_chemical_symbols())
金属掩码 = np.isin(元素数组, list(DEFAULT_METAL_SYMBOLS))
金属索引 = np.where(金属掩码)[0]

print(f"步骤1：筛选金属原子")
print(f"  DEFAULT_METAL_SYMBOLS 包含 {len(DEFAULT_METAL_SYMBOLS)} 种过渡金属")
print(f"  本体系包含: {set(元素数组[金属掩码])}")
print(f"  金属原子数量: {len(金属索引)}")

# 获取金属原子的 z 坐标（投影到 c 轴方向）
所有位置 = atoms.get_positions()
金属z坐标 = 所有位置[金属索引, 2]  # 简化：直接用 z 坐标

# 直方图：金属原子 z 坐标分布
fig, ax = plt.subplots(figsize=(8, 3))
ax.hist(金属z坐标, bins=80, color='tab:orange', edgecolor='none', alpha=0.8)
ax.set_xlabel('z 坐标 (Å)', fontsize=11)
ax.set_ylabel('金属原子数', fontsize=11)
ax.set_title('步骤1：金属原子 z 坐标分布（每个尖峰 = 一层）', fontsize=12)
plt.tight_layout()
plt.show()

print(f"\n观察：直方图中有 {len(all_layers)} 个尖峰，对应 {len(all_layers)} 层金属")

### 步骤 2：聚类（clustering）— 把相近 z 坐标的原子归为一层

**思路**：按 z 坐标排序后，如果相邻两个原子的 z 差值超过阈值（`layer_tol_A = 0.6 Å`），就认为它们属于不同层。

In [ ]:
# 模拟聚类步骤
排序索引 = np.argsort(金属z坐标)
金属z排序 = 金属z坐标[排序索引]

layer_tol_A = 0.6  # 聚类容差：同一层内原子 z 坐标相差不超过 0.6 Å

# 标记层边界（相邻原子差超过 tol 就是新层的开始）
差值 = np.diff(金属z排序)
边界 = np.where(差值 > layer_tol_A)[0] + 1  # 边界位置（新层开始的索引）

# 构建层：每个层包含的原子 z 坐标
层列表 = np.split(金属z排序, 边界)

print(f"步骤2：聚类结果（容差 {layer_tol_A} Å）")
print(f"  检测到 {len(层列表)} 层金属")
print()
for i, 层 in enumerate(层列表):
    print(f"  层 {i}: {len(层)} 个原子, z 范围 [{层.min():.2f}, {层.max():.2f}] Å, 中心 {层.mean():.2f} Å")

# 可视化聚类结果
fig, ax = plt.subplots(figsize=(10, 3))

颜色 = plt.cm.tab10(np.linspace(0, 0.9, len(层列表)))
for i, (层, 色) in enumerate(zip(层列表, 颜色)):
    ax.scatter(层, [1]*len(层), c=[色], s=15, label=f'层{i}', zorder=3)
    ax.axvline(x=层.mean(), color=色, linewidth=2, alpha=0.5)

ax.set_xlabel('z 坐标 (Å)', fontsize=11)
ax.set_yticks([])
ax.set_title('步骤2：聚类结果（每种颜色 = 一层金属原子）', fontsize=12)
ax.legend(fontsize=8, ncol=len(层列表), loc='upper center')
plt.tight_layout()
plt.show()

### 步骤 3：识别界面层 — 周期性边界条件下的「最大间隔」策略

**思路**：  
在周期性边界下，金属层均匀排列，金属板两侧是水。  
水区域（最大间隔）位于哪两层之间，这两层就是界面层。

```
沿 c 方向（分数坐标 0→1，周期性）：

层0  层1  层2  层3   [最大间隔 = 水区]   层0
 │    │    │    │   ←── 水区域 ──→   │
 └────┴────┴────┘                   │（周期）
      金属板区域                    界面!
```

In [ ]:
# 用分数坐标（分数坐标 = z坐标 / c晶胞长度，范围 0~1）
scaled = atoms.get_scaled_positions()
金属分数坐标 = scaled[金属索引, 2]  # 金属原子的 c 分数坐标

# 各层中心的分数坐标
层中心分数 = [层.mean() / c_A for 层 in 层列表]
层中心分数.sort()

# 计算相邻层之间的间隔（周期性，首尾也要考虑）
间隔列表 = []
for k in range(len(层中心分数)):
    f0 = 层中心分数[k]
    f1 = 层中心分数[(k+1) % len(层中心分数)]
    间隔 = (f1 - f0) % 1.0  # 周期性间隔
    间隔列表.append(间隔)

最大间隔索引 = int(np.argmax(间隔列表))
界面层low_idx  = 最大间隔索引
界面层high_idx = (最大间隔索引 + 1) % len(层中心分数)

print("步骤3：各层间隔（分数坐标）")
for k, (间隔, f) in enumerate(zip(间隔列表, 层中心分数)):
    标记 = " ←── 最大间隔（水区）" if k == 最大间隔索引 else ""
    print(f"  层{k}（f={f:.4f}) → 层{(k+1)%len(层中心分数)} 间隔={间隔:.4f}{标记}")

print(f"\n界面层: 层{界面层low_idx}（低侧）和 层{界面层high_idx}（高侧）")

# 可视化：在分数坐标圆上展示各层和最大间隔
fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(projection='polar'))
角度列表 = [f * 2 * np.pi for f in 层中心分数]

for k, (角度, f) in enumerate(zip(角度列表, 层中心分数)):
    if k in [界面层low_idx, 界面层high_idx]:
        色 = 'red'
        大小 = 150
        标签 = f'界面层 {k}'
    else:
        色 = 'tab:orange'
        大小 = 80
        标签 = f'层 {k}' if k == 0 else '_nolegend_'
    ax.scatter([角度], [1.0], s=大小, c=色, zorder=5, label=标签)
    ax.text(角度, 1.15, f'层{k}\nf={f:.3f}', ha='center', va='center', fontsize=9)

ax.set_rticks([])
ax.set_xticks([0, np.pi/2, np.pi, 3*np.pi/2])
ax.set_xticklabels(['f=0', 'f=0.25', 'f=0.5', 'f=0.75'])
ax.set_title('步骤3：各层在分数坐标圆上的位置\n（红点 = 界面层，间距最大处两侧）', fontsize=11)
ax.legend(loc='lower right', bbox_to_anchor=(1.35, -0.05), fontsize=9)

plt.tight_layout()
plt.show()

### 步骤 4：计算法向量 — metal → water 方向

In [ ]:
界面层 = 检测结果.interface_layers()

print("步骤4：各界面层的法向量")
print()
for layer in 界面层:
    nv = layer.normal_unit
    if nv[2] > 0:
        含义 = "朝 +z 方向（指向上侧水区）"
    else:
        含义 = "朝 -z 方向（指向下侧水区）"
    print(f"  center_s = {layer.center_s:.2f} Å")
    print(f"  normal_unit = {nv}")
    print(f"  含义: {含义}")
    print()

---

## 4.3 两个界面的概念

由于**周期性边界条件（PBC）**，金属板有两个界面（上下各一个），都接触水。  
本项目对两个界面分别进行分析，通常选其中一个（`start_interface` 参数控制）。

In [ ]:
# 可视化：整个模拟盒子的截面示意图
from src.structure.Analysis.WaterAnalysis._common import _detect_low_high_interface_fractions

low_c, high_c = _detect_low_high_interface_fractions(atoms)

print(f"两个界面的分数坐标：")
print(f"  low_c  = {low_c:.4f}  → z = {low_c * c_A:.3f} Å  （下界面）")
print(f"  high_c = {high_c:.4f}  → z = {high_c * c_A:.3f} Å  （上界面）")
print(f"  界面间距 = {(high_c - low_c) * c_A:.3f} Å")
print(f"  水区半宽 = {(high_c - low_c) * c_A / 2:.3f} Å  （从界面到中央）")

fig, ax = plt.subplots(figsize=(4, 10))

# 画晶胞边界
ax.set_xlim(0, 1)
ax.set_ylim(0, c_A)

# 水区（蓝色）
水区上 = [high_c * c_A, c_A]   # high_c 到晶胞顶（= low_c 区的另一侧）
水区下 = [0, low_c * c_A]      # 晶胞底到 low_c
ax.axhspan(水区上[0], 水区上[1], alpha=0.2, color='tab:blue', label='水区（上）')
ax.axhspan(水区下[0], 水区下[1], alpha=0.2, color='cyan',     label='水区（下，周期镜像）')

# 金属板（橙色）
ax.axhspan(low_c * c_A, high_c * c_A, alpha=0.4, color='tab:orange', label='金属板区')

# 画各金属层
for i, layer in enumerate(all_layers):
    # 注意：center_s 是投影坐标，对于正交晶胞等于 z 坐标
    ax.axhline(y=layer.center_s, color='black', linewidth=0.8, alpha=0.7)
    ax.text(0.5, layer.center_s + 0.2, f'金属层 {i}', ha='center', fontsize=8)

# 画界面线
ax.axhline(y=low_c * c_A,  color='red', linewidth=2.5, linestyle='--', label=f'low_c 界面 ({low_c:.3f})')
ax.axhline(y=high_c * c_A, color='darkred', linewidth=2.5, linestyle='--', label=f'high_c 界面 ({high_c:.3f})')

# 画法向量箭头
ax.annotate('', xy=(0.5, low_c * c_A - 2), xytext=(0.5, low_c * c_A),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax.text(0.55, low_c * c_A - 1.5, '法向量\n(→ 下侧水)', color='red', fontsize=8)

ax.annotate('', xy=(0.5, high_c * c_A + 2), xytext=(0.5, high_c * c_A),
            arrowprops=dict(arrowstyle='->', color='darkred', lw=2))
ax.text(0.55, high_c * c_A + 0.8, '法向量\n(→ 上侧水)', color='darkred', fontsize=8)

# 标注中点
中点z = (low_c + high_c) * c_A / 2
ax.axhline(y=中点z, color='green', linewidth=1, linestyle=':', label='金属板中点')

ax.set_xlabel('（横向位置，示意图）', fontsize=10)
ax.set_ylabel('z 坐标 (Å)', fontsize=11)
ax.set_title('模拟盒子截面图\n（显示金属层和两个界面）', fontsize=12)
ax.legend(loc='upper right', fontsize=8)
ax.set_xticks([])

plt.tight_layout()
plt.show()

---

## 本章小结

| 概念 | 说明 |
|------|------|
| `detect_interface_layers(atoms)` | 自动检测所有金属层，标记界面层 |
| `Layer.center_s` | 层中心的 z 坐标（Å） |
| `Layer.is_interface` | 是否是界面层（直接接触水） |
| `Layer.normal_unit` | 界面法向量，指向水的方向 |
| `low_c` / `high_c` | 两个界面层在 c 轴的分数坐标 |
| 聚类策略 | 按 z 坐标排序后，间隔 > 0.6 Å 即分层 |
| 界面识别策略 | 分数坐标圆上，间隔最大处两侧即为界面层 |

**下一章**（`05_layer1_water_analysis.ipynb`）将分析水分子的拓扑构建和单帧密度/取向计算。